<a href="https://colab.research.google.com/github/achintiii/MachineLearningWork/blob/main/Vehicle_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# Install necessary dependencies
!pip install ultralytics

import os
from ultralytics import YOLO

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Define paths to the data directories
train_images_dir = '/content/drive/My Drive/train/images'
train_labels_dir = '/content/drive/My Drive/train/labels'
valid_images_dir = '/content/drive/My Drive/valid/images'
valid_labels_dir = '/content/drive/My Drive/valid/labels'


In [41]:
import matplotlib.pyplot as plt
import cv2

In [42]:
# Create YAML file for the dataset
dataset_yaml_content = f"""
path: /content/drive/My Drive

train: train/images
val: valid/images

nc: 5  # number of classes
names: ['car', 'truck', 'ambulance', 'motorcycle', 'other']  # class names
augment:
  flipud: 0.5  # Randomly flip images vertically
  fliplr: 0.5  # Randomly flip images horizontally
  hsv_h: 0.015  # Image HSV-Hue augmentation (fraction)
  hsv_s: 0.7  # Image HSV-Saturation augmentation (fraction)
  hsv_v: 0.4  # Image HSV-Value augmentation (fraction)
  translate: 0.1  # Image translation (fraction)
  scale: 0.5  # Image scale (fraction)
  shear: 0.0  # Image shear (fraction)
  perspective: 0.0  # Image perspective (fraction)
  rotate: 0.0  # Image rotation (degrees)
  mosaic: 1.0  # Mosaic augmentation probability
  mixup: 0.5  # Mixup augmentation probability
  cutmix: 0.5  # CutMix augmentation probability
"""

In [43]:
# Save the YAML file
yaml_file_path = '/content/drive/My Drive/dataset.yaml'
with open(yaml_file_path, 'w') as file:
    file.write(dataset_yaml_content)

In [44]:
# Define the custom Focal Loss function
import torch
import torch.nn as nn
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce_loss = nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        ce_loss = self.ce_loss(logits, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

In [45]:
model = YOLO('yolov8n.pt')

In [46]:
# Initial Training Phase
model.train(data=yaml_file_path, epochs=10)

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        250        454      0.725      0.536      0.594      0.451



10 epochs completed in 0.024 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.23it/s]


                   all        250        454      0.727       0.54      0.594      0.451
                   car         50         64      0.835      0.789      0.868      0.743
                 truck         30         46      0.806      0.543      0.617      0.511
             ambulance         90        238      0.557      0.479      0.476      0.335
            motorcycle         42         46      0.702      0.522      0.569       0.32
                 other         38         60      0.733      0.367       0.44      0.347
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2d83bf1f90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [47]:
training_params = {
    'data': yaml_file_path,
    'epochs': 50,  # Adjust based on your needs
    'batch': 16,  # Adjust based on your GPU capabilities
    'imgsz': 640,  # Image size; larger images may yield better results
    'lr0': 0.01,  # Starting learning rate; fine-tune this parameter
    'optimizer': 'SGD',  # Experiment with different optimizers like 'Adam'
    'patience': 10,  # Early stopping patience to prevent overfitting
    'weight_decay': 0.0005,  # Regularization term to reduce overfitting
    'project': '/content/drive/My Drive/yolo_training',  # Save logs and weights to Google Drive
    'name': 'yolov8_experiment',  # Experiment name
    'save_period': 10  # Save weights every 10 epochs
}

In [48]:
# Train the model on the dataset
model.train(**training_params)

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/My Drive/dataset.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_experiment5, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, s

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_experiment5/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_experiment5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.407      0.431      0.368      0.231
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



11 epochs completed in 0.036 hours.
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_experiment5/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_experiment5/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/yolo_training/yolov8_experiment5/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.35it/s]


                   all        250        454      0.705      0.502      0.574      0.433
                   car         50         64      0.758      0.812      0.863      0.733
                 truck         30         46      0.745      0.522      0.583      0.463
             ambulance         90        238      0.673      0.387       0.49      0.324
            motorcycle         42         46      0.683      0.391      0.561      0.369
                 other         38         60      0.664        0.4      0.373      0.278
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 6.8ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training/yolov8_experiment5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2d89970970>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [49]:
# Define the directory path
save_dir = '/content/drive/My Drive/yolov8_experiment'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model after training
model.save(os.path.join(save_dir, 'best_model.pt'))

In [50]:
# Step 3: Load the saved model for further improvement
model = YOLO('/content/drive/My Drive/yolov8_experiment/best_model.pt')


In [51]:
# Example: Fine-tuning with a lower learning rate and additional epochs
fine_tuning_params = {
    'data': yaml_file_path,  # Use the same or updated dataset
    'epochs': 20,  # Additional epochs for fine-tuning
    'batch': 16,  # Keep or adjust batch size based on your GPU
    'imgsz': 640,  # Image size; you can keep it the same or adjust
    'lr0': 0.005,  # Lower the learning rate for fine-tuning
    'optimizer': 'AdamW',  # Experiment with a different optimizer
    'patience': 10,  # Early stopping to prevent overfitting
    'weight_decay': 0.0005,  # Regularization to prevent overfitting
    'project': '/content/drive/My Drive/yolo_training',  # Same project path
    'name': 'yolov8_fine_tuning_experiment',  # New experiment name
    'save_period': 5,  # Save weights every 5 epochs
}

# Continue training the loaded model with new parameters
model.train(**fine_tuning_params)

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/My Drive/yolov8_experiment/best_model.pt, data=/content/drive/My Drive/dataset.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_fine_tuning_experiment18, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=F

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18/labels.jpg... 
optimizer: AdamW(lr=0.005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.599      0.511      0.534      0.397



20 epochs completed in 0.059 hours.
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18/weights/best.pt, 6.2MB

Validating /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18/weights/best.pt...
Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.00it/s]


                   all        250        454      0.607      0.523      0.541      0.409
                   car         50         64      0.749      0.875      0.875      0.746
                 truck         30         46      0.803      0.609      0.624      0.467
             ambulance         90        238      0.537      0.429      0.438      0.306
            motorcycle         42         46      0.548      0.391      0.461      0.301
                 other         38         60      0.399       0.31      0.308      0.227
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment18


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f2d83bf1210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [52]:
# Define the directory path
save_dir = '/content/drive/My Drive/yolov8_experiment'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model after training
model.save(os.path.join(save_dir, 'model#2.pt'))

In [53]:
# Step 3: Load the saved model for further improvement
model = YOLO('/content/drive/My Drive/yolov8_experiment/model#2.pt')



In [54]:
dataset_yaml_content = f"""
path: {save_dir}

train: train/images
val: valid/images

nc: 5  # number of classes
names: ['car', 'truck', 'ambulance', 'motorcycle', 'other']  # class names

# Augmentation strategies (emphasizing trucks and other classes)
augment:
  flipud: 0.5  # Flip vertically with a probability of 50%
  fliplr: 0.5  # Flip horizontally with a probability of 50%
  hsv_h: 0.015  # HSV Hue augmentation (range: 0-0.5)
  hsv_s: 0.7  # HSV Saturation augmentation (range: 0-1.0)
  hsv_v: 0.4  # HSV Value augmentation (range: 0-1.0)
  translate: 0.1  # Translate images (range: 0-0.5)
  scale: 0.5  # Scale images (range: 0.5-2.0)
  shear: 0.1  # Shear images (range: 0-0.5)
  perspective: 0.1  # Apply perspective transformation (range: 0-0.5)
  rotate: 10.0  # Rotate images by 10 degrees
  mosaic: 1.0  # Use Mosaic augmentation with 100% probability
  mixup: 0.5  # Increase Mixup augmentation to create harder examples
  cutout: 0.5  # Use Cutout to remove parts of the image and force learning
  'class_weights': [1, 1.5, 1.5, 2.5, 3],  # Further increase weights for Ambulance and Other classes
  'cos_lr': True,  # Implement Cosine Annealing Learning Rate
"""

# Save the YAML file again with updated augmentations
yaml_file_path = os.path.join(save_dir, 'dataset_improved.yaml')
with open(yaml_file_path, 'w') as file:
    file.write(dataset_yaml_content)

In [55]:
# Continue training the loaded model with the new parameters and augmented dataset
model.train(**fine_tuning_params)

# Save the fine-tuned model after additional training
model.save(os.path.join(save_dir, 'fine_tuned_model_final.pt'))

Ultralytics YOLOv8.2.76 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/My Drive/yolov8_experiment/model#2.pt, data=/content/drive/My Drive/dataset.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=/content/drive/My Drive/yolo_training, name=yolov8_fine_tuning_experiment19, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

train: Scanning /content/drive/My Drive/train/labels.cache... 878 images, 0 backgrounds, 0 corrupt: 100%|██████████| 878/878 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/My Drive/valid/labels.cache... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<?, ?it/s]


Plotting labels to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment19/labels.jpg... 
optimizer: AdamW(lr=0.005, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /content/drive/My Drive/yolo_training/yolov8_fine_tuning_experiment19
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        250        454      0.194      0.167      0.113     0.0685


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2efed60af0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1443, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/20       2.3G      1.272      1.729      1.499         58        640:   7%|▋         | 4/55 [00:02<00:25,  1.98it/s]


KeyboardInterrupt: 

In [56]:
model = YOLO('/content/drive/My Drive/yolov8_experiment/fine_tuned_model_final.pt')

In [57]:
# Define the path to your test images directory
test_images_dir = '/content/drive/My Drive/test/images'
test_labels_dir = '/content/drive/My Drive/test/labels'

In [58]:
# Get list of all images in the test directory
test_images = [img for img in os.listdir(test_images_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

In [61]:


# Function to display image with predictions
def display_image_with_predictions(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display with matplotlib

    # Run prediction on the image
    results = model(image_path)

    # Extract predictions in a format we can display
    predictions = []
    for i, bbox in enumerate(results[0].boxes.xyxy):  # Access the bounding boxes
        x1, y1, x2, y2 = map(int, bbox[:4].tolist())  # Convert to integers
        conf = results[0].boxes.conf[i].item()  # Get confidence score
        label = model.names[int(results[0].boxes.cls[i].item())]  # Get the label

        predictions.append({
            'box': [x1, y1, x2, y2],  # Bounding box coordinates
            'confidence': conf,  # Confidence score
            'label': label  # Class label
        })

    # Draw bounding boxes and labels on the image
    for pred in predictions:
        x1, y1, x2, y2 = pred['box']
        conf = pred['confidence']
        label = pred['label']

        # Draw rectangle
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Put label and confidence
        cv2.putText(img_rgb, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the image with bounding boxes and labels
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

for image in test_images:
    image_path = os.path.join(test_images_dir, image)
    display_image_with_predictions(image_path)


image 1/1 /content/drive/My Drive/test/images/00e481ea1a520175_jpg.rf.6e6a8b3b45c9a11d106958f88ff714ea.jpg: 640x640 2 motorcycles, 8.7ms
Speed: 2.6ms preprocess, 8.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/00dea1edf14f09ab_jpg.rf.KJ730oDTFPdXdJxvSLnX.jpg: 640x640 1 car, 9.3ms
Speed: 2.7ms preprocess, 9.3ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/00dea1edf14f09ab_jpg.rf.3f17c8790a68659d03b1939a59ccda80.jpg: 640x640 1 car, 8.9ms
Speed: 2.7ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/00e481ea1a520175_jpg.rf.MV6sZ8QCFwFeMYaI2tHm.jpg: 640x640 2 motorcycles, 8.3ms
Speed: 2.5ms preprocess, 8.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/10c26c6598677a1f_jpg.rf.USCbBYVcUICkLhuq07Lw.jpg: 640x640

<ipython-input-61-c3ea06f8f6e4>:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 10))



image 1/1 /content/drive/My Drive/test/images/3429c6851095a4c3_jpg.rf.fqPa3xt3eK2RXoQ756m6.jpg: 640x640 2 ambulances, 8.3ms
Speed: 2.5ms preprocess, 8.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/3429c6851095a4c3_jpg.rf.2921b5c17bda3bda8b69e0f8b5e44894.jpg: 640x640 2 ambulances, 8.5ms
Speed: 2.5ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/479012386aa308fb_jpg.rf.24d80cdb95dae89bc61b36651306c45f.jpg: 640x640 3 motorcycles, 9.1ms
Speed: 2.9ms preprocess, 9.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/4ecafc9a0bae8003_jpg.rf.f9f2a4cdc2c519b8feeb3ab0c04a7e1d.jpg: 640x640 1 ambulance, 1 other, 9.4ms
Speed: 3.1ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /content/drive/My Drive/test/images/64918de984156adf_jpg.rf.